In [5]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import joblib
import optuna  # For hyperparameter tuning

# Tickers
tickers = ["BNB-USD", "BTC-USD", "ETH-USD", "XRP-USD"]

# Prepare data with feature engineering
def prepare_data(df):
    features = ["Open", "High", "Low", "Close", "Volume", "Lag1", "Lag7", "SMA7", "RSI14", "MACD", "Returns", "Volatility"]
    X = df[features].values
    y = df["Target"].values
    return X, y, features

# Objective function for Optuna hyperparameter tuning
def objective(trial, X_train_scaled, y_train, X_test_scaled, y_test):
    params = {
        'kernel': trial.suggest_categorical('kernel', ['rbf', 'poly']),
        'C': trial.suggest_float('C', 1e-1, 1e3, log=True),
        'epsilon': trial.suggest_float('epsilon', 0.01, 0.2, log=True),
        'gamma': trial.suggest_float('gamma', 1e-4, 1e-1, log=True) if trial.params['kernel'] == 'rbf' else None
    }
    
    if params['kernel'] == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 5)
    
    model = SVR(**{k: v for k, v in params.items() if v is not None})
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Train and evaluate SVR model
for ticker in tickers:
    print(f"\nTraining SVR for {ticker}...")
    
    # Load data
    df = pd.read_csv(f"{ticker}_processed.csv", index_col="Date", parse_dates=True)
    X, y, feature_names = prepare_data(df)
    
    # Split data (80% train, 20% test, no shuffle for time series)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # Scale features
    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    
    # Hyperparameter tuning with Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X_train_scaled, y_train, X_test_scaled, y_test), n_trials=30)
    best_params = study.best_params
    print(f"Best parameters for {ticker}: {best_params}")
    
    # Train final model with best parameters
    model = SVR(**best_params)
    model.fit(X_train_scaled, y_train)
    
    # Predict and evaluate
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    test_dates = df.index[train_size:]
    actual_series = pd.Series(y_test, index=test_dates)
    pred_series = pd.Series(y_pred, index=test_dates)
    actual_direction = (actual_series.shift(-1) > actual_series).iloc[:-1].astype(int)
    predicted_direction = (pred_series.shift(-1) > pred_series).iloc[:-1].astype(int)
    directional_accuracy = (actual_direction == predicted_direction).mean() * 100
    
    print(f"SVR Results for {ticker}:")
    print(f"MSE: {mse:.2f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"Directional Accuracy: {directional_accuracy:.2f}%")
    
    # Save model and scaler
    joblib.dump(model, f"{ticker}_svr_model.pkl")
    joblib.dump(scaler_X, f"{ticker}_scaler_X_svr.pkl")
    print(f"Model and scaler saved for {ticker}")

[I 2025-03-31 23:52:45,967] A new study created in memory with name: no-name-22509d79-8e47-430d-9799-5d9a8f3808e8
[I 2025-03-31 23:52:45,991] Trial 0 finished with value: 65075.35045336339 and parameters: {'kernel': 'rbf', 'C': 24.215300629197802, 'epsilon': 0.07571476481696401, 'gamma': 0.00019888613979533406}. Best is trial 0 with value: 65075.35045336339.
[I 2025-03-31 23:52:45,998] Trial 1 finished with value: 32127.598981576924 and parameters: {'kernel': 'poly', 'C': 0.6077137943777243, 'epsilon': 0.023560422359011474, 'degree': 3}. Best is trial 1 with value: 32127.598981576924.
[I 2025-03-31 23:52:46,024] Trial 2 finished with value: 105265.26854211031 and parameters: {'kernel': 'rbf', 'C': 0.23217146524291424, 'epsilon': 0.04091959599138519, 'gamma': 0.0036358289877444993}. Best is trial 1 with value: 32127.598981576924.
[I 2025-03-31 23:52:46,053] Trial 3 finished with value: 1425.3864105796056 and parameters: {'kernel': 'rbf', 'C': 153.32228925156585, 'epsilon': 0.19546059005


Training SVR for BNB-USD...


[I 2025-03-31 23:52:46,168] Trial 7 finished with value: 1738647.8919128713 and parameters: {'kernel': 'poly', 'C': 26.945807619004352, 'epsilon': 0.08351872092248, 'degree': 5}. Best is trial 3 with value: 1425.3864105796056.
[I 2025-03-31 23:52:46,193] Trial 8 finished with value: 343871.03819712717 and parameters: {'kernel': 'poly', 'C': 0.5616434805294154, 'epsilon': 0.018656955993946043, 'degree': 5}. Best is trial 3 with value: 1425.3864105796056.
[I 2025-03-31 23:52:46,208] Trial 9 finished with value: 103188.58824206781 and parameters: {'kernel': 'poly', 'C': 2.636670807572105, 'epsilon': 0.06293714832848188, 'degree': 3}. Best is trial 3 with value: 1425.3864105796056.
[I 2025-03-31 23:52:46,280] Trial 10 finished with value: 3359.198899147318 and parameters: {'kernel': 'rbf', 'C': 740.9286299151175, 'epsilon': 0.19611900540661925, 'gamma': 0.08544883781806578}. Best is trial 3 with value: 1425.3864105796056.
[I 2025-03-31 23:52:46,346] Trial 11 finished with value: 3689.50061

Best parameters for BNB-USD: {'kernel': 'rbf', 'C': 965.7969823616656, 'epsilon': 0.05626105909101916, 'gamma': 0.0005316736572457543}
SVR Results for BNB-USD:
MSE: 365.04
MAPE: 2.15%
Directional Accuracy: 47.14%
Model and scaler saved for BNB-USD

Training SVR for BTC-USD...


[I 2025-03-31 23:52:47,100] Trial 7 finished with value: 2513582821.6099267 and parameters: {'kernel': 'rbf', 'C': 0.5884028349017179, 'epsilon': 0.011506518906496363, 'gamma': 0.002368667013103353}. Best is trial 5 with value: 2308469984.442553.
[I 2025-03-31 23:52:47,115] Trial 8 finished with value: 2372058801.7102075 and parameters: {'kernel': 'poly', 'C': 0.2446486316128303, 'epsilon': 0.023010402306755565, 'degree': 3}. Best is trial 5 with value: 2308469984.442553.
[I 2025-03-31 23:52:47,134] Trial 9 finished with value: 2359699980.681679 and parameters: {'kernel': 'poly', 'C': 0.26600980351970754, 'epsilon': 0.03353634054417955, 'degree': 3}. Best is trial 5 with value: 2308469984.442553.
[I 2025-03-31 23:52:47,150] Trial 10 finished with value: 3326730044982.4136 and parameters: {'kernel': 'poly', 'C': 179.39823779033767, 'epsilon': 0.10258671899773973, 'degree': 5}. Best is trial 5 with value: 2308469984.442553.
[I 2025-03-31 23:52:47,182] Trial 11 finished with value: 137651

Best parameters for BTC-USD: {'kernel': 'poly', 'C': 419.2673607037677, 'epsilon': 0.07126984576134512, 'degree': 2}
SVR Results for BTC-USD:
MSE: 191248768.58
MAPE: 13.61%
Directional Accuracy: 50.00%
Model and scaler saved for BTC-USD

Training SVR for ETH-USD...


[I 2025-03-31 23:52:47,802] Trial 8 finished with value: 6014126.467788986 and parameters: {'kernel': 'poly', 'C': 10.891683754841097, 'epsilon': 0.10274540403165626, 'degree': 5}. Best is trial 3 with value: 36690.38559966695.
[I 2025-03-31 23:52:47,824] Trial 9 finished with value: 51976.10899718381 and parameters: {'kernel': 'rbf', 'C': 25.258279322233964, 'epsilon': 0.069755905128118, 'gamma': 0.020529790330975902}. Best is trial 3 with value: 36690.38559966695.
[I 2025-03-31 23:52:47,864] Trial 10 finished with value: 50738.06036390688 and parameters: {'kernel': 'rbf', 'C': 781.8175997942255, 'epsilon': 0.19367128213366058, 'gamma': 0.09892684395863684}. Best is trial 3 with value: 36690.38559966695.
[I 2025-03-31 23:52:47,906] Trial 11 finished with value: 48670.2531554412 and parameters: {'kernel': 'rbf', 'C': 932.0764334988392, 'epsilon': 0.19450909853492618, 'gamma': 0.08848330290283309}. Best is trial 3 with value: 36690.38559966695.
[I 2025-03-31 23:52:47,938] Trial 12 finis

Best parameters for ETH-USD: {'kernel': 'rbf', 'C': 411.21862217547084, 'epsilon': 0.014200988890497313, 'gamma': 0.004324469559256763}
SVR Results for ETH-USD:
MSE: 14603.80
MAPE: 2.99%
Directional Accuracy: 50.00%
Model and scaler saved for ETH-USD

Training SVR for XRP-USD...


[I 2025-03-31 23:52:48,749] Trial 7 finished with value: 713201.774099751 and parameters: {'kernel': 'poly', 'C': 1.6751501293689643, 'epsilon': 0.011442820741477969, 'degree': 3}. Best is trial 0 with value: 1.9305026551849802.
[I 2025-03-31 23:52:48,749] Trial 8 finished with value: 1.8926470550995729 and parameters: {'kernel': 'rbf', 'C': 78.7938723825132, 'epsilon': 0.0915952837311376, 'gamma': 0.0001996524388112983}. Best is trial 8 with value: 1.8926470550995729.
[I 2025-03-31 23:52:48,769] Trial 9 finished with value: 2.6175039789414116 and parameters: {'kernel': 'rbf', 'C': 1.957719208459868, 'epsilon': 0.01107942375513707, 'gamma': 0.034961351661089886}. Best is trial 8 with value: 1.8926470550995729.
[I 2025-03-31 23:52:48,790] Trial 10 finished with value: 0.7237644103747306 and parameters: {'kernel': 'rbf', 'C': 28.254241873352015, 'epsilon': 0.039522412254342935, 'gamma': 0.0001284630463194091}. Best is trial 10 with value: 0.7237644103747306.
[I 2025-03-31 23:52:48,798] T

Best parameters for XRP-USD: {'kernel': 'rbf', 'C': 28.254241873352015, 'epsilon': 0.039522412254342935, 'gamma': 0.0001284630463194091}
SVR Results for XRP-USD:
MSE: 0.72
MAPE: 27.02%
Directional Accuracy: 52.14%
Model and scaler saved for XRP-USD
